## メタプログラミング
プログラムを使ってコードの生成や書き換えなどを行うことをメタプログラミングとする。Julia以外のプログラミング言語では、LispのマクロやPythonのデコレータなどが例になる。

Juliaはメタプログラミングを比較的多く用いる言語である。**メタプログラミングを使用することで、複雑なコードを実行時に生成したり、冗長なコードの反復を回避することができる**。@assertなど、@から始まるマクロの呼び出しから成り立ち、**受け取ったJuliaのコードをオブジェクトや別のコードに変換**する。また、r"\d+"のような正規表現などで知られている非標準文字列リテラルも使われる。これもマクロの一種であり、**受け取った文字列をオブジェクトや別のコードへと変換**する。

### シンボル
シンボルはJuliaが取り扱うデータ型の一種で、処理系の内部で使われる名前(識別子)に対応する。例えば、fooという変数をソースコードで使用すると、そのコードを構文解析したときにfooという名前のシンボルが処理系の内部に作られる。

**名前の前にコロンをつけることで作成することができる**

In [1]:
println(:foo)

typeof(ans)

foo


Nothing

In [2]:
# 以下のようにSymbol型のコンストラクタを呼び出して作ることもできる
Symbol("foo")
# コンストラクタの引数は文字列や数値、他のシンボルを渡して結合した名前を作り出すこともできる
Symbol("foo", :bar, 9)

:foobar9

### 構文木(abstract syntax tree: AST)

入力されたコードやファイルに保存されたコードは単なる文字列だが、コードはJuliaの処理系に読み込まれると構文解析され、抽象構文木と呼ばれるデータ構造に変換される。抽象構文木は単に構文木と呼ばれる。

構文木はJuliaのオブジェクトとして扱うことができる。:(hoge)でJuliaのコードを囲むと、Juliaの処理系はそのコードを実行せずに構文木をオブジェクトとして取り出す。この操作をクォートと呼ぶ

In [3]:
ex = :(2x+1)

:(2x + 1)

一行の単純なコードのクォートは:()を用いるが、複数行からなるコードのクォートにはquoteキーワードを使用する。quoteとendで挟まれた部分のコードは、:(hoge)で囲んだコードと同じようにクォートされる。

整数や文字列などのリテラルを除けば、構文木はExpr型で表現されるオブジェクト型になる(expressionの短縮形)。**Expr型のオブジェクトにはheadフィールドとargsフィールドがあり、headフィールドが構文木の種類を表し、argsフィールドがその構成要素を保持している**。

以下の例では、2x+1という計算式の内部構造をdump関数で観察している。この構文木のheadフィールドは:callというシンボルが収められており、構文木が関数呼び出しであることを意味している。argsフィールドには三つの要素が収められており、1要素目が:+シンボル、2要素目が2xに対応する別の構文木、3要素目が整数1のリテラルである。

In [8]:
dump(ex)

Expr
  head: Symbol call
  args: Array{Any}((3,))
    1: Symbol +
    2: Expr
      head: Symbol call
      args: Array{Any}((3,))
        1: Symbol *
        2: Int64 2
        3: Symbol x
    3: Int64 1


In [9]:
dump(:(x=10))

Expr
  head: Symbol =
  args: Array{Any}((2,))
    1: Symbol x
    2: Int64 10


### 構文木の補間
クォートで作られた構文木には、別の構文木やリテラルを埋め込める。これを**補間(interpolation)**と呼ぶ。**構文は文字列の補間と同じように\$記号を用いる**

$(func(ex))のように関数を適用した結果で補間することもできる

In [12]:
ex = :(3y + 1);
:(2x + $(ex))

:(2x + (3y + 1))

メタプログラミングにおいては、シンボルは変数などの識別子として使われることがほとんどなので、シンボルのリテラルでなく識別子として補間される方が利便性が高い。シンボルのリテラルとして補間する必要がある場合には、以下のようにQuoteNodeでクォートする。

In [13]:
ex = :y;
:(2x + $(ex))

:(2x + y)

In [14]:
:(2x + $(QuoteNode(ex)))

:(2x + :y)

### 構文木の評価

eval関数で評価(evaluation)できる。評価とは、構文木を実行して結果を得ることである。eval関数は構文木を引数として受け取り、現在のモジュールでその構文木を実行して結果を返す。

ただし、評価は常にモジュールのグローバルスコープで行われるので注意する必要がある。例えば、関数の中でeval関数を呼び出し、構文木の評価を行ったとしても、その構文木はローカルスコープにある変数とは干渉しない(二つ目の例)

test関数の例では、最初に書かれているx="local"は関数ないのローカル変数を定義しているが、次の行のeval関数に渡されたx="global"はMainモジュールで実行されてグローバル変数xを定義しているので、ローカル変数xを上書きすることはできない。

In [1]:
x = 10;
eval(:(2x + 1))

21

In [3]:
function test()
    x = "local"
    eval(:(x = "global"))
    println(x)
end

test (generic function with 1 method)

In [4]:
test()

local


In [5]:
x

"global"

eval関数を使用することで、簡単で便利なメタプログラミングが可能になる。例えば、ビットフラグを持つ三つの定数を導入するのにfor文とeval関数を使って評価することが可能になる。

In [6]:
for (i, name) in enumerate([:A, :B, :C])
    eval(:(const $(Symbol(:FLAG_, name))=
            $(UInt16(1) << (i-1))
    ))
end

FLAG_A, FLAG_B, FLAG_C

(0x0001, 0x0002, 0x0004)

eval関数と同様の機能を提供する@evalマクロも、メタプログラミングに便利である。@evalマクロは与えられたコードをクウォートして、その結果をeval関数で評価する。よって、eval関数では必要なクォートを省略できる。

In [8]:
for (i, name) in enumerate([:A, :B, :C])
    @eval const $(Symbol(:FLAG_, name)) = $(UInt16(i) << (i-1))
end

### マクロの機能

**マクロは、与えられたコードを別のコードに変換してから実行するための仕組みである**。

Juliaの重要な機能の一つであり、マクロを使うときは関数のようにマクロを呼び出すが、その構文は関数の呼び出しとは異なり@記号を用いる。

展開の結果は@macroexpandマクロを使って確認することができる。これによって、**与えらえれた式にあるマクロ呼び出しを展開して構文木を返す**。例えば、条件が満たされているかを確認する@assertマクロの呼び出しを展開した結果は、以下のようなif文を含む形になる。

In [9]:
@macroexpand @assert x>0

:(if x > 0
      nothing
  else
      Base.throw(Base.AssertionError("x > 0"))
  end)

マクロ呼び出しの展開は、コンパイルのかなり早い段階で行われる。展開はJuliaのコンパイラがソースコードを構文解析した直後に行われる。よって、**マクロは処理するコードの識別子がどのような型や値になるか全く考慮せずに、構文木のレベルでのみコードの変換を行う**。

マクロの呼び出しには@macro(ex1, ex2, ex3)と括弧を用いる呼び出し方と、@macro ex1 ex2 ex3のように括弧を用いない呼び出し方の二つがある。両者は構文解析の仕方が違うだけで機能に差はない。**慣例として、括弧は省略する形式が多い**。


In [11]:
# @macroはREPLで確認することができる
#(@macro x + y) == :(@macro(x + y))

普通のコードを受け取るマクロの他に、**文字列のみを受け取るマクロ**も存在する。これらの特殊なマクロは、**非標準文字列リテラル**と呼ばれている。

こちらの場合はマクロ名が_strの形で終わり、通常の文字列の前に_strを除いたマクロ名をつけて呼び出せる。

### 標準ライブラリにあるマクロ

主にJuliaの標準ライブラリで提供されているマクロには、四種類ある。

・コンパイラへのヒント：　@inbounds, @inline, @fastmath など

・構文の拡張： @assert, @enum, @view など

・開発の補助： @less, @time, @code_typed など

・特殊なリテラル: @r_str, @big_str など

**コンパイラへのヒントになるマクロは、最適化などのヒントになる情報を構文木に特殊なデータを差し込むことでコンパイラへ渡すマクロになる**。@inboundsは、配列要素の参照が配列の有効な範囲に収まることをプログラマが保証するので、実行時に範囲のチェックを省いても良いことになる。@inlineマクロは関数を積極的にインライン化するべきというヒントを与える。@fastmathマクロは浮動小数点の計算に関してIEEE754の制約を超えて最適化することを許可するマクロになる。

**構文の拡張のマクロは、プログラマが手で書くには面倒な処理を自動化するマクロになる**。例えば、@assertマクロは与えられた式の条件が成立するかどうかを実行時にチェックして、その条件が成立しないとAssertionError例外を返す。@enumマクロはCのenum構文に相当する機能を提供する。@viewマクロは、X[i,:]などの配列の一部をコピーする構文を、コピーではなくその部分の参照として作るようにする。

**開発の補助のマクロは、REPLなどの環境で用いられるマクロになる**。**特殊なリテラルは、非標準文字列リテラルとして機能して、オブジェクトを作るのに使用される**。


### マクロの定義

Juliaでは、関数やデータ型と同様に、ユーザが新しいマクロを定義することができる。マクロの定義にはmacroキーワードを使用する。

例えば、計算式の最後に+1を付与するだけの@plus1マクロを定義する。試しに構文木の補間を使って、以下のように定義する。

In [18]:
macro plus1(ex)
    :($(ex) + 1)
end

@plus1 (macro with 1 method)

In [19]:
# xはMainモジュールのグローバル変数xになっていることに注意すること
@macroexpand @plus1 2x

:(2 * Main.x + 1)

上記のようにグローバル変数にしたくない場合は、**esc関数を使うことで識別子がグローバル変数に変換されないようにすることができる**。

In [22]:
macro plus1(ex)
    :($(esc(ex)) + 1)
end    

@plus1 (macro with 1 method)

In [23]:
@macroexpand @plus1 2x

:(2x + 1)

### 識別子の変換規則

マクロ展開では、識別子はグローバル変数に変換されるのがデフォルトの動作だったが、ローカル変数に変換される例外もある。例えば、

・global宣言なしで代入されたとき

・local宣言があるとき

・関数定義の引数があるとき

例えば、マクロが返す構文木にx=0があれば変数xはローカル変数となる。また、local xとしてる場合も同様にローカル変数として扱われる。

また、**ローカル変数と解釈された識別子は、マクロ展開時に新しい変数に置き換えられる**。これはマクロ呼び出し側にある別の識別子との衝突を避けるためでもある。このようなマクロ展開のやり方を**衛生的(hygienic)マクロ**と呼ぶ。実際にコードの実行時間をns秒単位で計測する@time_nsマクロを定義して効果を確認してみる。

In [24]:
macro time_ns(ex)
    quote
        t1 = time_ns()
        val = $(esc(ex))
        t2 = time_ns()
        val, Int(t2-t1)
    end
end

@time_ns (macro with 1 method)

In [25]:
# 計算結果と実行時間のタプルを返す
# 2回目以降は計算が高速になる
@time_ns sum(randn(1000))

(-11.421643460216425, 153443583)

In [26]:
@time_ns sum(randn(1000))

(41.67640484189498, 54458)

もし、時間を計測したいコードexにt1 = ...のような代入があった場合、どうなるか。衛生的マクロではなく、ローカル変数をそのまま維持するマクロだったとしたら、マクロ定義ないのローカル変数t1 = time_ns()の結果を上書きしてしまう。こうなると、目的の時間計測が不可能になってしまう。

マクロ定義中のt1, t2, valはglobal宣言なしで代入されているので、ローカル変数と解釈される。衛生的マクロでは、これらのローカル変数は\#10\#t1などの奇妙な名前の新しいローカル変数に置換される。結果的にローカル変数は衝突しない。

In [27]:
@macroexpand @time_ns sum(randn(1000))

quote
    #= In[24]:3 =#
    var"#9#t1" = Main.time_ns()
    #= In[24]:4 =#
    var"#10#val" = sum(randn(1000))
    #= In[24]:5 =#
    var"#11#t2" = Main.time_ns()
    #= In[24]:6 =#
    (var"#10#val", Main.Int(var"#11#t2" - var"#9#t1"))
end

まとめると、マクロ内の識別子がいつどのように変換されるか、あるいは変換されないかをまとめて確認する。マクロが返す構文木やリテラルに含まれる識別子は次のいずれかの経路を辿る。

```
・esc関数でエスケープされていれば、識別子は変換されずそのまま維持される

・代入、local宣言、関数引数のいずれかであれば、新しいローカル変数が生成される。

・上記のどれにも当てはまらない場合、マクロを定義したモジュールのグローバル変数に変換される。
```

## 外部プログラムの呼び出し

### コマンドの作成・実行

Juliaで外部プログラムを使用するには、PerlやRubyと同様にバッククォート記号を用いる。例えば、\`ls\`とする

In [4]:
# REPLと異なり、jupyter上ではrunの中にコマンドを入れる必要性がある
run(`ls`)

Introduction1.ipynb
Introduction2.ipynb
Introduction3.ipynb
Introduction4.ipynb
README.md
sample.jl


Process(`ls`, ProcessExited(0))

In [38]:
# コマンドの標準出力を得る場合にはread関数を使うのが良い
read(`ls`, String)

"Introduction1.ipynb\nIntroduction2.ipynb\nIntroduction3.ipynb\nIntroduction4.ipynb\nREADME.md\nsample.jl\n"

また、eachline関数を使って標準出力を業ごとに処理できる。Cmd型のオブジェクトは他の関数と組み合わせて使用できるので柔軟性が高い。

In [39]:
for line in eachline(`find . -type f`)
    @show line
end

line = "./sample.jl"
line = "./README.md"
line = "./Introduction4.ipynb"
line = "./.gitignore"
line = "./Introduction2.ipynb"
line = "./.ipynb_checkpoints/Introduction4-checkpoint.ipynb"
line = "./.ipynb_checkpoints/Introduction1-checkpoint.ipynb"
line = "./.ipynb_checkpoints/Introduction2-checkpoint.ipynb"
line = "./.ipynb_checkpoints/Introduction3-checkpoint.ipynb"
line = "./.git/config"
line = "./.git/objects/0d/9f3df8c2ec4624a8271f125acda0aaf5bad8e4"
line = "./.git/objects/95/33569d32c86e02724d4d2fd1990e80b3e8f804"
line = "./.git/objects/50/c88d404d83f6e6249e9049aca33c0cb399e5c1"
line = "./.git/objects/69/fd7671cc2569d268c1169c8eb604a5d2308592"
line = "./.git/objects/0b/11152410238d591fb4d7d8d441a17768b55756"
line = "./.git/objects/34/07b032a599848ac7149011187e30e1188a831b"
line = "./.git/objects/02/5ff62e1506d770113fb20beebe13583cbee476"
line = "./.git/objects/ae/a4cc931074a3a41701b01f322f1d71c133377f"
line = "./.git/objects/ca/147d92e41c421c8ea99a2de805f81b55e1e6ab"
line = "./.gi

コマンドのプロセスからデータを受け取るだけでなく、プロセスに対してJuliaからデータを渡すことも可能である。

例えば起動したプロセスの標準入力に対して書き込みを行たい場合、次のようにopen関数を使うことで書き込むことができる。

In [41]:
open(`wc -l`, "w", stdout) do output
    for _ in 1:10
        println(output, "hi!")
    end
end

      10


### コマンド実行の注意点

Juliaのコマンドは、一般的なシェルを経由せずに実行される。よって、パターンマッチに使われる*やパイプライン処理|はそのままコマンドの引数として扱われるので、bashなどのシェルとは異なった動作をする。

シェルの機能が必要な場合には、明示的にシェルを執行できる。POSIXに準拠しているbashコマンドには-cオプションがあるので、これに実行したいコマンドの文字列を渡すと、シェルを経由してコマンドを実行することができる(ローカルのMACはbashを入れてないので実行できないので注意汗)。

また、パイプライン処理(pipeline)は複数コマンドの入力と出力を接続する機能である。

In [5]:
#run(`bash -c 'ls *.txt'`)

Introduction1.ipynb
Introduction2.ipynb
Introduction3.ipynb
Introduction4.ipynb
README.md
sample.jl


Process(`ls`, ProcessExited(0))